# FEB-II: data-taking

Take 100 micro bunches and check for missing words. 

Questions: what is the expected format for start of first hit with physical channels?


Referring to https://github.com/Mu2e/CRV_FEB2/blob/main/README.md 

In [32]:
crv.febPwrCycle()

Power cycling all FEB ports


In [63]:
# External imports 
import sys
import numpy as np
import time
# Internal imports
sys.path.append("..")
import CRV 

In [64]:
# Initialise CRV class
crv = CRV.CRV("/dev/ttyUSB0", True) # 2nd ROC is USB0

## Read/write helper functions from Simon

In [65]:
def r(add):
    add_ = add if isinstance(add, str) else str(add)
    return crv.read(add_, lc=True)[0]
def w(add, val):
    add_ = add if isinstance(add, str) else str(add)
    val_ = val if isinstance(val, str) else str(val)
    return crv.write(add_, val_, lc=True)

## Soft reset

Confused about this. Sometimes it works sometimes not. Seems to be fine after a power cycle? I struggle to get sensible looking data without this.

In [66]:
# w("000", "0020") # Set bit 5 for soft reset 
# w("000", "0000") # Clear bits

## Enable self-trigger mode

```
0x303 	Trigger Control Register
	Bit 0 = Self Trigger Enable (SlfTrgEn)
		0 = use FM encoded clock from ROC to generate timing signals (default)
		1 = use local free running counters to generate "fake" timing signals
	Bit 1 = VCXO 160MHz clock control ("TrgSrc") 
		0 = frequency lock the 160MHz clock to the ROC clock (default)
		1 = disable VCXO frequency locking to the ROC clock and 
		    force the VCXO control voltage to stable mid-point.	

	The fake timing signals are a crude approximation of a super cycle and 
	consist of 25k "on spill" windows of 1700ns followed by 10k "off spill"
	windows of 100us, repeats every 1.0425 seconds.
```

In [67]:
# Enable self-triggered mode
w("303", "0001")  # Broadcast write
# w("303", "0000")  # Broadcast write

send:  LC WR 303 0001
readback:  b'LC WR 303 0001\r\r\n'


## Map channels 

```
0x080-0x08F Input mux control registers map physical channels to logical channels

0x080 Logical channel  0 mux control (default value = 0)
...
0x08F Logical channel 15 mux control (default value = 15)

	Each of these 5 bit registers is R/W and 
	the register contents are defined as:

	0 = AFE0 ch 0		8  = AFE1 ch 0
...
	7 = AFE0 ch 7		15 = AFE1 ch 7

	16 = fake positive going pulse aligned with start of livegap
	     this pulse starts at 0, sharp rise up to 0xAAA, then decay 
	     back to zero. for details see pulser.vhd
```

In [68]:
def map_channels(fake=True):
    # Map all logical channels 
    for i in range(16):
        reg = f"0{i:X}" if i < 10 else f"{i:X}"  # Format: 00, 01, ..., 0F 
        if fake: 
            w(f"8{reg}", "0010") # map logical channel to test (fake) channel 16
        else:
            w(f"8{reg}", f"{reg}") # map logical channel to physical channels

map_channels(
    fake=False # fake channel or physical channels
)

send:  LC WR 800 00
readback:  b'LC WR 800 00\r\r\n'
send:  LC WR 801 01
readback:  b'LC WR 801 01\r\r\n'
send:  LC WR 802 02
readback:  b'LC WR 802 02\r\r\n'
send:  LC WR 803 03
readback:  b'LC WR 803 03\r\r\n'
send:  LC WR 804 04
readback:  b'LC WR 804 04\r\r\n'
send:  LC WR 805 05
readback:  b'LC WR 805 05\r\r\n'
send:  LC WR 806 06
readback:  b'LC WR 806 06\r\r\n'
send:  LC WR 807 07
readback:  b'LC WR 807 07\r\r\n'
send:  LC WR 808 08
readback:  b'LC WR 808 08\r\r\n'
send:  LC WR 809 09
readback:  b'LC WR 809 09\r\r\n'
send:  LC WR 8A A
readback:  b'LC WR 8A A\r\r\n'
send:  LC WR 8B B
readback:  b'LC WR 8B B\r\r\n'
send:  LC WR 8C C
readback:  b'LC WR 8C C\r\r\n'
send:  LC WR 8D D
readback:  b'LC WR 8D D\r\r\n'
send:  LC WR 8E E
readback:  b'LC WR 8E E\r\r\n'
send:  LC WR 8F F
readback:  b'LC WR 8F F\r\r\n'


## Check output FIFO 
```
0x017: 	Core Output FIFO Status 
	(note this register replaces the multi-FPGA broadcast read 0x317) R/O
	bit 4: output FIFO empty flag 
	bit 0: always 1
```

In [69]:
r("017")

send:  LC RD 017
readback:  b'LC RD 017\r\r\n'
read : b'0011\r\n'


'0011'

## Setup data requests and readback 

```
0x312 	Request UB number, HIGH order bits 31..16

0x313 	Request UB number, LOW order bits 15..0
	Writing to this register will initiate a read cycle from the 
	DDR memory controller and a short time later the requested event
	data will appear in the core output FIFO.

...

The microcontroller requests an event to be read from the DDR memory by writing the UB number to regs 0x312 and 0x313. 
(Always write 0x312 FIRST, as writing to 0x313 is the trigger to GO!)

Keep reading words until 0x017 = 011

```

In [73]:
# is fifo empty helper
def fifo_empty():
    """Helper to check if output FIFO is empty"""
    # result = r("017")
    result = crv.read("017", lc=True)
    if result and len(result) > 0: # list index of out of range errors?
        return int(result[0][2]) == 1 # check bit 4 of 0x17
        # return int(result[2]) == 1 # check bit 4 of 0x17
    else:
        # Handle the empty result 
        print("Warning: read returned None or empty list")
        return False # Continue read

def reshape(data, pad_value="dead"):
    """Helper to reshape the data into eight EB columns""" 
    # Convert to list 
    data = list(data)
    
    # Check if we need padding
    remainder = len(data) % 8 
    if remainder > 0:
        padding_needed = 8 - remainder
        data.extend([pad_value] * padding_needed)
        
    # Convert to numpy array
    data = np.array(data)
    # Number of rows of eight 
    rows = len(data) // 8 # Floor division
    # Truncate the array to only include complete rows 
    # (removing remainder elements)
    # data = data[:rows*8] # no need if padding
    # Reshape 
    data = data.reshape(rows, 8)
    # Return
    return data
    
def get_data(uB=0):
    """Request a uB, read back the FIFO, return reshaped data""" 
    
    v = crv.verbose # Store original verbosity
    crv.verbose = 0 # Reduce verbosity
            
    if not fifo_empty():
        crv.readm("c", 2**10, lc=True) # flash out all remaining data first
    
    # request data
    w(312, "0")
    w(313, hex(uB)[2:]) # convert to hex and strip "0x"
    # w(313, format(uB, "x")) # Is this better? Seems harder to read

    # Wait a bit
    time.sleep(0.1) 
    
    # Read out FIFO 
    while not fifo_empty():
        time.sleep(0.1)
        fifo_data = crv.readm("C", n=(2**10), lc=True)
    
    crv.verbose = v # restore verbosity 

    # Reshape into rows x 8 
    fifo_data = reshape(fifo_data)
    
    # Return
    return fifo_data

## Take data

In [10]:
# print(f"FIFO empty? {fifo_empty()}")

In [74]:
n_samples = 100

def take_data(n=n_samples):  
    uB_samples = {} 
    for uB in range(n):
        uB_samples[uB] = get_data(uB)
    
    if not fifo_empty():
        print("Warning: FIFO not empty")

    return uB_samples

uB_samples = take_data(n_samples)

## Write out formatted data

In [75]:
uB_samples[0]

array([['000a', '000a', '0009', '000a', '0004', '000a', '000a', '0004'],
       ['0000', '0002', '0005', '0006', '0008', '000a', '000c', '000e'],
       ['0012', '0012', '0012', '0012', '0012', '0012', '0012', '0016'],
       ['ffb4', 'efb6', '9ff3', '7fea', 'dfe5', '9fe9', '2fe3', '3003'],
       ['c1fb', 'c4fb', 'e3fe', 'e4fe', 'ecfe', 'e5fe', 'e0fe', 'fe00'],
       ['fc2f', 'fc1f', 'fd9f', 'fdcf', 'fe8f', 'fe5f', 'fe6f', 'ffbf'],
       ['6fc6', '2fb6', 'ffcf', '7fe1', 'dfe2', '8fe8', '3fe7', '4008'],
       ['cafc', 'b0fb', 'd6fc', 'f4fe', 'e1fd', 'f4fe', 'dbfe', '1601'],
       ['fd1f', 'fb5f', 'fe8f', '000f', 'fe8f', 'ff5f', 'fe3f', '0160'],
       ['bfd3', '0fba', '0ff2', '7ffe', 'cff5', 'eff9', '0fef', '1009'],
       ['befc', 'a6fb', 'e8ff', 'f2ff', 'f6ff', 'e7fe', 'f2ff', '0600'],
       ['fb6f', 'fa3f', 'fe5f', 'fe9f', 'feff', 'fe0f', 'febf', '00a0'],
       ['0000', '0002', '0005', '0006', '0008', '000a', '000c', '000e'],
       ['002a', '002a', '002a', '002a', '002a', '00

In [76]:
def write_data(samples, out_file_name, printOut=False):

    # Column headers for event builders
    headers = ["EB0", "EB1", "EB2", "EB3", "EB4", "EB5", "EB6", "EB7"]
    
    # Create header line
    header_line = "     " + " ".join(f"{h:>5}" for h in headers)
    separator_line = "     " + "-" * (5*8 + 7)  # Separator line

    # Write
    with open(out_file_name, "w") as f:

        for uB, sample in samples.items():

            # Format microbunch number in decimal and hex
            # uB_hex = hex(uB)[2:].upper().zfill(2) 
            uB_hex = format(uB, 'X').zfill(2)
            
            # Write header to file
            f.write(f"Microbunch: {uB:3d} (0x{uB_hex})\n\n")
            f.write(header_line + "\n")
            f.write(separator_line + "\n")
            
            # Write data to file and print to console
            if printOut or uB == 0:
                print(f"Microbunch: {uB:3d} (0x{uB_hex})\n\n")
                print(header_line + "\n")
                print(separator_line + "\n")
            
            for i, row in enumerate(sample):
                if len(row) == 8:  # Make sure we have a complete row
                    formatted_row = f"{i:3d}:  {' '.join(f'{val:>5}' for val in row)}"
                    f.write(formatted_row + "\n")
                    if printOut or uB == 0:
                        print(formatted_row)

            f.write(separator_line + "\n\n")
            if printOut or uB == 0:
                print(separator_line + "\n\n")
                
        print(f"\n\tWrote file {out_file_name}")

write_data(uB_samples, f"feb2_dumps/feb2_{n_samples}uBs_physical_2.txt")

Microbunch:   0 (0x00)


       EB0   EB1   EB2   EB3   EB4   EB5   EB6   EB7

     -----------------------------------------------

  0:   000a  000a  0009  000a  0004  000a  000a  0004
  1:   0000  0002  0005  0006  0008  000a  000c  000e
  2:   0012  0012  0012  0012  0012  0012  0012  0016
  3:   ffb4  efb6  9ff3  7fea  dfe5  9fe9  2fe3  3003
  4:   c1fb  c4fb  e3fe  e4fe  ecfe  e5fe  e0fe  fe00
  5:   fc2f  fc1f  fd9f  fdcf  fe8f  fe5f  fe6f  ffbf
  6:   6fc6  2fb6  ffcf  7fe1  dfe2  8fe8  3fe7  4008
  7:   cafc  b0fb  d6fc  f4fe  e1fd  f4fe  dbfe  1601
  8:   fd1f  fb5f  fe8f  000f  fe8f  ff5f  fe3f  0160
  9:   bfd3  0fba  0ff2  7ffe  cff5  eff9  0fef  1009
 10:   befc  a6fb  e8ff  f2ff  f6ff  e7fe  f2ff  0600
 11:   fb6f  fa3f  fe5f  fe9f  feff  fe0f  febf  00a0
 12:   0000  0002  0005  0006  0008  000a  000c  000e
 13:   002a  002a  002a  002a  002a  002a  002a  0082
 14:   efb9  2faa  dfe7  2fe1  6fea  bfdb  1feb  500f
 15:   c6fb  b9fb  f7fe  effe  f0fe  dffd  fcff  fb00
 16

In [77]:
uB_samples[0][1]

array(['0000', '0002', '0005', '0006', '0008', '000a', '000c', '000e'],
      dtype='<U7')

In [67]:
# hit_start = []
        
# # Find first hit start (first row with all '0010')
# for i in range(0, len(uB_samples[0])):
#     print(i, list(uB_samples[0][i]))
#     if (list(uB_samples[0][i]) == ["0000",  "0002",  "0004",  "0006",  "0008",  "000a",  "000c",  "000e"]):
#         hit_start.append(i)
# hit_start

In [78]:
def ana_fake_chan(uB_samples):
    """
    Check if uB data matches expected structure using the fake channel
    
    Expected structure:
    - Header row with upper nibbles of first 4 words (lower 32 bits) reflect the microbunch number
    - Two hits, each 11 rows long, starting with '0010' (when using fake channel 16)
    - Garbage (ffff) after exactly two hits in self-trigger mode

    """
    invalid_samples = {}
    
    for uB, sample in uB_samples.items(): 
        # Container for issues
        issues = []  

        # 0. Check if we have enough rows for two hits
        if len(sample) < 23:
            issues.append(f"Too few rows for two hits {len(data)}, expected at least 23")
        
        # 1. Check uB number
        expected_uB_number = hex(uB)[2:]
        header = sample[0]
        uB_data = [word[0] for word in header[:4]] # First char of first four words
        found_uB_number = "".join(list(reversed(uB_data)))
        uB_number_correct = (int(expected_uB_number, 16) == int(found_uB_number, 16))

        if not uB_number_correct:
            issues.append(f"uB # incorrect: expected: {expected_uB_number}, found: {found_uB_number}")

        # Find positions of key markers
        first_hit_start = -1
        second_hit_start = -1
        garbage_start = -1
        
        # Find first hit start (first row with all '0010')
        for i in range(1, len(data)):
            if all(word == '0010' for word in data[i]):
                first_hit_start = i
                break
        
        # Find second hit start (next row with all '0010' after first hit)
        if first_hit_start != -1:
            for i in range(first_hit_start + 1, len(data)):
                if all(word == '0010' for word in data[i]):
                    second_hit_start = i
                    break
        
        # Find garbage start (first row with all 'ffff')
        for i in range(1, len(data)):
            if all(word == 'ffff' for word in data[i]):
                garbage_start = i
                break
                
        # 2. Check first hit starts with '0010'
        if not all(word == '0010' for word in data[1]):
            issues.append("First hit does not start with '0010'")
        
        # 3. Check second hit starts with '0010'
        if not all(word == '0010' for word in data[12]):
            issues.append("Second hit does not with '0010'")
        
        # 4. Check garbage data starts after second hit
        if len(data) >= 23:
            # Check if row 23 has 'ffff'
            if not all(word == 'ffff' for word in data[23]):
                issues.append("Garbage (ffff) does not start after second hit")

        # Fancier methods are available
        
        # Add issues to invalid samples 
        if len(issues) > 0:
            invalid_samples[uB] = issues

    return invalid_samples

def ana_phys_chan(uB_samples):
    """
    Check if uB data matches expected structure using physical channels
    
    Expected structure:
    - Header row with upper nibbles of first 4 words (lower 32 bits) reflect the microbunch number
    - N hits, each 11 rows long, starting with [0000,  0002,  0005,  0006,  0008,  000a,  000c,  000e] (do we understand this format? I thought it was the channel numbers) 
    - Garbage (ffff) after N hits 

    """
    invalid_samples = {}
    
    for uB, sample in uB_samples.items(): 
        # Container for issues
        issues = []  
        
        # 1. Check uB number
        expected_uB_number = hex(uB)[2:]
        header = sample[0]
        uB_data = [word[0] for word in header[:4]] # First char of first four words
        found_uB_number = "".join(list(reversed(uB_data)))
        uB_number_correct = (int(expected_uB_number, 16) == int(found_uB_number, 16))

        if not uB_number_correct:
            issues.append(f"uB # incorrect: expected: {expected_uB_number}, found: {found_uB_number}")

        # Find positions of key markers
        garbage_start = -1
        hit_start = []
        
        # Find hit starts
        # Not sure I understand the expected format
        # I'm pretty sure it inidcates the channel mapping, but why do we skip channel 4 sometimes? 
        for i in range(1, len(sample)):
            this_sample = list(sample[i])
            if ( 
                (this_sample == ["0000",  "0002",  "0004",  "0006",  "0008",  "000a",  "000c",  "000e"])
                or 
                (this_sample == ["0000",  "0002",  "0005",  "0006",  "0008",  "000a",  "000c",  "000e"])
            ):
                hit_start.append(i)

        # Find number of words between hits 
        n_words = np.diff(hit_start)
        if not all(n_words == 11):
            issues.append(f"Truncated hits found: {n_words}")
            
        
        # Find garbage start (first row with all 'ffff')
        for i in range(1, len(sample)):
            if all(word == 'ffff' for word in sample[i]):
                garbage_start = i
                break

        if garbage_start == -1:
            issues.append(f"Garbage never starts. Corrupt last hit?")

        if all(word == 'ffff' for word in sample[0]):
            issues.append(f"uB is garbage")
            
        # Fancier methods are available
        
        # Add issues to invalid samples 
        if len(issues) > 0:
            invalid_samples[uB] = issues

    return invalid_samples

def get_analysis(uB_samples, fake=True):
    """
    Provides a summary of the validation results for all samples.
    """
    invalid_samples = {} 
    if fake:
        invalid_samples = ana_fake_chan(uB_samples)
    else: 
        invalid_samples = ana_phys_chan(uB_samples)
    
    return {
        "total_samples": len(uB_samples),
        "valid_samples": len(uB_samples) - len(invalid_samples),
        "invalid_samples": len(invalid_samples),
        "invalid_sample_uBs": list(invalid_samples.keys()),
        "detailed_issues": invalid_samples
    }

# invalid_samples = analyse_data(uB_samples)
analysis = get_analysis(uB_samples, fake=False)
# print(analysis)

# print("*"*75)
# print("Summary (100 uBs):")
# for i, k in analysis.items():
#     print(i, k)
# print("*"*75)
# {60: ['uB # incorrect: expected: 3c, found: ffff']}

## Printout analysis (overcooked formatting from AI)

In [79]:
# Massively overcooked formatting using AI

def print_analysis_summary(analysis, total_uBs=n_samples):
    print(f"\n{'-'*60}")
    print(f"  Summary ({total_uBs} uBs)")
    print(f"{'-'*60}")
    
    # Print basic statistics
    print(f"  Total samples:     {analysis['total_samples']}")
    print(f"  Valid samples:     {analysis['valid_samples']} ({analysis['valid_samples']/analysis['total_samples']*100:.1f}%)")
    print(f"  Invalid samples:   {analysis['invalid_samples']} ({analysis['invalid_samples']/analysis['total_samples']*100:.1f}%)")
    
    # Print info about invalid samples if any
    if analysis['invalid_samples'] > 0:
        print(f"\n  Invalid uBs:")
        print(f"  {'-'*40}")
        
        # Format the list of invalid uBs more nicely
        if len(analysis['invalid_sample_uBs']) <= 10:
            # If few invalid samples, list them all with their hex values
            invalid_uBs = [f"{uB} (0x{format(uB, 'X').zfill(2)})" for uB in analysis['invalid_sample_uBs']]
            print(f"  {', '.join(invalid_uBs)}")
        else:
            # If many invalid samples, just list the count
            print(f"  {len(analysis['invalid_sample_uBs'])} microbunches have issues")
        
        # Show detailed issues for each invalid sample
        print(f"\n  Issues:")
        print(f"  {'-'*40}")
        for uB, issues in analysis['detailed_issues'].items():
            print(f"  uB {uB} (0x{format(uB, 'X').zfill(2)}):")
            for issue in issues:
                print(f"    - {issue}")
    
    # Add a nice completion message
    print(f"{'-'*60}")
    if analysis['invalid_samples'] == 0:
        print(f"  ✅ All uBs OK")
    else:
        print(f"  ⚠️ {analysis['invalid_samples']} uBs failed")
    print(f"{'-'*60}\n")

print_analysis_summary(analysis)


------------------------------------------------------------
  Summary (100 uBs)
------------------------------------------------------------
  Total samples:     100
  Valid samples:     0 (0.0%)
  Invalid samples:   100 (100.0%)

  Invalid uBs:
  ----------------------------------------
  100 microbunches have issues

  Issues:
  ----------------------------------------
  uB 0 (0x00):
    - Garbage never starts. Corrupt last hit?
  uB 1 (0x01):
    - Garbage never starts. Corrupt last hit?
  uB 2 (0x02):
    - Garbage never starts. Corrupt last hit?
  uB 3 (0x03):
    - Garbage never starts. Corrupt last hit?
  uB 4 (0x04):
    - Garbage never starts. Corrupt last hit?
  uB 5 (0x05):
    - Garbage never starts. Corrupt last hit?
  uB 6 (0x06):
    - Garbage never starts. Corrupt last hit?
  uB 7 (0x07):
    - Garbage never starts. Corrupt last hit?
  uB 8 (0x08):
    - Garbage never starts. Corrupt last hit?
  uB 9 (0x09):
    - Garbage never starts. Corrupt last hit?
  uB 10 (0x0A)

In [64]:
# Good uBs? 
analysis.keys()
# valid_uBs = [i if not in analysis["invalid_sample_uBs"] for i in range(500)]
valid_uBs = [i for i in range(500) if i not in analysis["invalid_sample_uBs"]]
valid_uBs

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,